In [1]:
!pip install python-chess
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import glob
import os
import chess


  Obtaining dependency information for chess<2,>=1 from https://files.pythonhosted.org/packages/d6/d8/15cfcb738d2518daf04d34b23419bd359cbd8e09da50778ebac521774fc8/chess-1.10.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 5.9 MB/s eta 0:00:00


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

!pip install kaggle
os.environ['KAGGLE_CONFIG_DIR'] = '/content'
!kaggle datasets download -d mallikarjunreddy3015/chessfendataset

 99% 143M/144M [00:08<00:00, 25.0MB/s]
100% 144M/144M [00:08<00:00, 18.2MB/s]


In [3]:
import psutil

# Get system memory information
memory = psutil.virtual_memory()
print(f"Total RAM: {memory.total / (1024 ** 3):.2f} GB")
print(f"Used RAM: {memory.used / (1024 ** 3):.2f} GB")
print(f"Free RAM: {memory.free / (1024 ** 3):.2f} GB")


Total RAM: 58.87 GB
Used RAM: 1.29 GB
Free RAM: 55.76 GB


In [4]:
import zipfile

# Replace 'your-dataset.zip' with the actual zip file name
with zipfile.ZipFile('/content/chessfendataset.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/chessfendataset')


In [5]:
dataframe = pd.read_csv('/content/chessfendataset/chessData.csv')

In [6]:
!nvidia-smi


Wed Oct  4 17:40:46 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8               9W /  70W |      2MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
dataframe.head()
d=dataframe

In [8]:
d.shape
d1=d.iloc[0:5000000 ,:]
d1.shape
d1 = d1[~d1['Evaluation'].str.contains('\ufeff|#', na=False)]
d1['Evaluation'] = pd.to_numeric(d1['Evaluation'].str.replace('[^\d.-]', '', regex=True), errors='coerce')

In [9]:
d1['Scaled_Evaluation'] = 1000 * (2 * (d1['Evaluation'] +7000) / (14000) - 1)

In [10]:
i=0
def create_bitboard(fen):
    i+1
    # Setting up the piece values
    piece_values = {
        'P': 11, 'p': -11, 'N': 44, 'n': -44, 'B': 33, 'b': -33, 'R': 55,
        'r': -55, 'Q': 99, 'q': -99, 'K': 110, 'k': -110
    }

    # Mapping piece symbols to bitboard layers
    piece_to_layer = {
        'P': 0, 'p': 1, 'N': 2, 'n': 3, 'B': 4, 'b': 5, 'R': 6, 'r': 7, 'Q': 8,
        'q': 9, 'K': 10, 'k': 11
    }

    # Create an empty bitboard of shape (12, 8, 8) using numpy
    en_passant_layer = 12
    white_values_layer = 13
    black_values_layer = 14

    bitboard = np.zeros((15, 8, 8))

    board = chess.Board(fen)
    if board.ep_square is not None:
        rank, file = divmod(board.ep_square, 8)
        bitboard[en_passant_layer][rank][file] = 1 if board.turn == chess.WHITE else -1
    computed_moves = {}

    for piece_type, piece_value in piece_values.items():
        layer = piece_to_layer[piece_type]

        if piece_type not in computed_moves:
            valid_moves = set()
            empty_board_moves = set()
            for square in chess.SQUARES:
                piece = board.piece_at(square)
                if piece and piece.symbol() == piece_type:
                    # Generate moves for both colors irrespective of turn
                    board.turn = chess.WHITE
                    moves_white = [move for move in board.legal_moves if move.from_square == square]
                    board.turn = chess.BLACK
                    moves_black = [move for move in board.legal_moves if move.from_square == square]
                    valid_moves.update([move.to_square for move in moves_white])
                    valid_moves.update([move.to_square for move in moves_black])

                    empty_board = chess.Board()
                    empty_board.clear()
                    empty_board.set_piece_at(square, piece)
                    # Generate moves for both colors irrespective of turn on the empty board
                    empty_board.turn = chess.WHITE
                    empty_moves_white = [move for move in empty_board.generate_legal_moves(from_mask=chess.BB_SQUARES[square])]
                    empty_board.turn = chess.BLACK
                    empty_moves_black = [move for move in empty_board.generate_legal_moves(from_mask=chess.BB_SQUARES[square])]
                    empty_board_moves.update([move.to_square for move in empty_moves_white])
                    empty_board_moves.update([move.to_square for move in empty_moves_black])

            computed_moves[piece_type] = (valid_moves, empty_board_moves)

        valid_moves, empty_board_moves = computed_moves[piece_type]

        for square in chess.SQUARES:
            rank, file = divmod(square, 8)
            piece = board.piece_at(square)
            if piece:
                if piece.color == chess.WHITE:
                    bitboard[white_values_layer][rank][file] = piece_values[piece.symbol()]
                else:
                    bitboard[black_values_layer][rank][file] = piece_values[piece.symbol()]
            if piece and piece.symbol() == piece_type:
                bitboard[layer][rank][file] = piece_value
            elif square in valid_moves:
                bitboard[layer][rank][file] = piece_value / 11
            elif square in empty_board_moves:
                bitboard[layer][rank][file] = piece_value / 22
        if i%100000==0:
           print(i)
        return bitboard

bitboard_dataset = [create_bitboard(fen) for fen in np.array(d1['FEN'])]
# print(generate_bitboard_with_moves(d1["FEN"][1]))


In [11]:
print(bitboard_dataset[20])

[[[   0.     0.     0.     0.     0.     0.     0.     0. ]
  [  11.    11.     0.     0.     0.     0.    11.    11. ]
  [   1.     1.     0.     0.     0.     0.     1.     1. ]
  [   1.     0.5    0.    11.     0.    11.     1.     1. ]
  [   0.     0.     0.     0.5   11.     1.     0.     0. ]
  [   0.     0.     0.     0.     0.5    1.     0.     0. ]
  [   0.     0.     0.     0.     0.     0.     0.     0. ]
  [   0.     0.     0.     0.     0.     0.     0.     0. ]]

 [[   0.     0.     0.     0.     0.     0.     0.     0. ]
  [   0.     0.     0.     0.     0.     0.     0.     0. ]
  [   0.     0.     0.     0.     0.     0.     0.     0. ]
  [   0.     0.     0.    -0.5    0.     0.     0.     0. ]
  [  -1.    -0.5    0.   -11.    -1.    -1.    -1.    -1. ]
  [  -1.    -0.5    0.     0.   -11.   -11.    -1.    -1. ]
  [ -11.   -11.     0.     0.     0.     0.   -11.   -11. ]
  [   0.     0.     0.     0.     0.     0.     0.     0. ]]

 [[   0.     0.     4.     0.     2.

In [12]:
# labels=d1['Evaluation']
labels=d1['Scaled_Evaluation']

In [ ]:
np.save('/content/bitboard_dataset_5M.npy', bitboard_dataset)
np.save('/content/labels_dataset_5M.npy', labels)

# bitboard_dataset=np.load("/content/chess-numpy-dataset/bitboard_dataset_1M.npy")
# labels=np.load("/content/chess-numpy-dataset/labels_dataset.npy")

In [ ]:
!pip install google-cloud-storage
import os

# Replace 'path/to/service-account-key.json' with the path to your service account key file
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/gleaming-modem-400906-3780658b0f47.json'

from google.cloud import storage

# Replace 'your-bucket-name' with the name of your Google Cloud Storage bucket
bucket_name = 'chess_data_all'

# Save the array to a binary file
file_path = '/content/labels_dataset_5M.npy'

# Create a Google Cloud Storage client
client = storage.Client()

# Specify the destination blob (file) in your bucket
blob_name = 'labels_dataset_5M.npy'

# Upload the file to the bucket
bucket = client.get_bucket(bucket_name)
blob = bucket.blob(blob_name)
blob.upload_from_filename(file_path)

print(f"File uploaded to: gs://{bucket_name}/{blob_name}")


In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv2D, Conv3D, Flatten, Dense, Concatenate
from tensorflow.keras.activations import relu
import tensorflow as tf
from tensorflow.keras.layers import DepthwiseConv2D


# ... Assuming bitboard_dataset and labels are defined ...

bitboard_dataset, labels = shuffle(bitboard_dataset, labels)
X_train, X_test, y_train, y_test = train_test_split(bitboard_dataset, labels, test_size=0.2, random_state=42)

# Radial Basis Function activation
def rbf(x):
    return tf.exp(-x**2)

# Input layer
input_layer_2d = layers.Input(shape=(15,8, 8))  # for 2D convolutions
input_layer_3d = layers.Reshape((15, 8, 8, 1))(input_layer_2d)  # for 3D convolutions

# Branch 1: Depthwise Convolution
branch1 = DepthwiseConv2D(kernel_size=(1, 1), depth_multiplier=1, activation=relu)(input_layer_2d)
branch1 = Conv2D(filters=15, kernel_size=(8, 8), activation=relu)(branch1)
branch1 = Flatten()(branch1)  # Flatten branch1
# Branch 2: Conv3D
branch2 = Conv3D(filters=16, kernel_size=(5, 5, 5), padding='valid', activation=relu)(input_layer_3d)
branch2 = Conv3D(filters=8, kernel_size=(3, 3, 3), padding='valid', activation=relu)(branch2)
branch2 = Flatten()(branch2)  # Flatten branch2

# Branch 3: Conv3 (Only to the 15th channel)
branch3_input = tf.gather(input_layer_2d, [13,14], axis=3)  # Extract the 15th channel
branch3 = DepthwiseConv2D(kernel_size=(1, 1), depth_multiplier=1, activation=relu)(branch3_input)
branch3 = Conv2D(filters=10, kernel_size=(8,8), activation=relu)(branch3)
branch3 = Conv2D(filters=2, kernel_size=(1, 1), activation=relu)(branch3)
branch3 = Flatten()(branch3)  # Flatten branch3

concat = Concatenate()([branch1, branch2, branch3])

# Dense layers
dense1 = Dense(64, activation='relu')(concat)
dense2 = Dense(32, activation='relu')(dense1)
dense3 = Dense(16, activation='relu')(dense2)
output_layer = Dense(1)(dense3)  # No activation function for regression

# Create the model
model = Model(inputs=input_layer_2d, outputs=output_layer)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'mape'])

# Train the model
model.fit(X_train, y_train, epochs=200, batch_size=1000, validation_split=0.1)

# Evaluate the model
loss, mae, mape = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 15, 8, 8, 1)]        0         []                            
                                                                                                  
 conv3d (Conv3D)             (None, 1, 1, 1, 8)           7688      ['input_1[0][0]']             
                                                                                                  
 conv3d_1 (Conv3D)           (None, 1, 4, 4, 4)           1504      ['input_1[0][0]']             
                                                                                                  
 conv3d_2 (Conv3D)           (None, 15, 1, 1, 8)          520       ['input_1[0][0]']             
                                                                                              

KeyboardInterrupt: ignored

In [ ]:

model.save('/content/drive/MyDrive/model1.keras')


FileNotFoundError: ignored